# Statistical Analysis of ERP

In [8]:
bids_root_path_100 = "/media/marc/Medien/xmasoddballmatch-bids"
pipeline_name_100 = "pipeline01"        
    
bids_root_path_150 = "/media/marc/Medien/machristine-bids"
pipeline_name_150 = "pipeline_christine"

In [10]:
from configuration import load_configuration

import utils

from os.path import join
import mne
from mne_bids import make_bids_basename, read_raw_bids
from mne_bids.utils import get_entity_vals

import plotting

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import ptitprince as pt
import pandas as pd
import matplotlib.gridspec as gridspec
import scipy as sp
from copy import deepcopy

config = load_configuration("../configuration/pipeline01_100.yml")

plt.ioff()


%matplotlib inline

In [6]:
def difference_wave(evokeds_as_dict, conditions, grandaverage=False):
    out = [mne.combine_evoked([a, b], [1,-1]) for a, b in zip(
        evokeds_as_dict[conditions[0]], evokeds_as_dict[conditions[1]])]
    if grandaverage:
        return mne.grand_average(out)
    else:
        return out

def get_mean_amplitudes(evokeds, window, picks = "all"):

    means = []
    if isinstance(evokeds, list):
        for i, evoked in enumerate(evokeds):
            evoked = evoked.copy().pick(picks)

            _window = np.arange(evoked.time_as_index(window[0]),
                                evoked.time_as_index(window[1]))

            data = evoked.data[:, _window]
            mean = data.mean()
            means.append(mean)
    else:
        return get_mean_amplitudes([evokeds], window, picks)[0]

    return means



## Load Data From Disk

In [11]:
%%capture
ids_100 = get_entity_vals(join(bids_root_path_100, "derivatives"), "sub") 
ave_filenames_100 = [utils.get_derivative_file_name(
        bids_root_path_100, id, pipeline_name_100, ".fif", suffix="ave") for id in ids_100]
all_evokeds_100 = [mne.read_evokeds(ave_filename) for ave_filename in ave_filenames_100]

ids_150 = get_entity_vals(join(bids_root_path_150, "derivatives"), "sub") 
ave_filenames_150 = [utils.get_derivative_file_name(
        bids_root_path_150, id, pipeline_name_150, ".fif", suffix="ave") for id in ids_150]
all_evokeds_150 = [mne.read_evokeds(ave_filename) for ave_filename in ave_filenames_150]

evokeds_list_as_dict = {}

evokeds_list_as_dict["100"] = {key: [] for key in config["conditions_of_interest"]}

for evokeds_list in all_evokeds_100:
    for evoked in evokeds_list:
        try:
            evokeds_list_as_dict["100"][evoked.comment].append(evoked)
        except KeyError:
            pass
            print("Missing condition in list.")

evokeds_list_as_dict["150"] = {key: [] for key in config["conditions_of_interest"]}

for evokeds_list in all_evokeds_150:
    for evoked in evokeds_list:
        try:
            evokeds_list_as_dict["150"][evoked.comment].append(evoked)
        except KeyError:
            pass
            print("Missing condition in list.")

In [13]:
# Find peak and find a window (±25ms) 
diff = difference_wave(evokeds_list_as_dict["100"], ("random/deviant", "random/standard"), grandaverage=True)
peak_latency = diff.pick(picks="FZ").get_peak(tmin = .1, tmax = .2,  return_amplitude = True)[1]

peakwindow = (peak_latency-0.025, peak_latency+0.025)

print("Peak Latency is {} s.".format(peak_latency))

Identifying common channels ...
Peak Latency is 0.13671875 s.


## Create DataFrame containing Mean Amplitudes

In [7]:
conditions       = {("100", "random", "B"): "random/5/deviant",
                    ("100", "random", "A"): "random/4/standard",
                    ("100", "predictable", "B"): "predictable/5/deviant",
                    ("100", "predictable", "A"): "predictable/4/standard",

                    ("150", "random", "B"): "random/5/deviant",
                    ("150", "random", "A"): "random/4/standard",
                    ("150", "predictable", "B"): "predictable/5/deviant",
                    ("150", "predictable", "A"): "predictable/4/standard"}

electrodes = {"FZ":"FZ", "CZ":"CZ", "M1": "M1", "M2": "M2", "fronto_pooled": ["FZ", "F3", "F4", "FC1", "FC2"], "mastoids_pooled": ["M1", "M2"]}

amplitudes = [{ "SOA":key[0], "Participant":key[0] + "_" + str(i), 
                "Condition":key[1], "StimulusType":key[2], 
                "MeanAmplitude": amplitude,
                "Electrode": electrode} 
                    for key,value in conditions.items() 
                    for electrode, pick in electrodes.items()
                    for i, amplitude in enumerate(get_mean_amplitudes(evokeds_list_as_dict[key[0]][value], peakwindow, picks=pick)) ]
amplitudes_df = pd.DataFrame(amplitudes)

amplitudes_df.to_csv("../data/mean_amplitudes.csv", index=False)

amplitudes_df


,SOA,Participant,Condition,StimulusType,MeanAmplitude,Electrode
0,100,100_0,random,B,-7.880820e-07,FZ
1,100,100_1,random,B,4.523228e-07,FZ
2,100,100_2,random,B,-8.314268e-07,FZ
3,100,100_3,random,B,-6.018624e-07,FZ
4,100,100_4,random,B,-2.535743e-06,FZ
...,...,...,...,...,...,...
1027,150,150_18,predictable,A,-3.963710e-08,mastoids_pooled
1028,150,150_19,predictable,A,7.267545e-09,mastoids_pooled
1029,150,150_20,predictable,A,-1.082195e-06,mastoids_pooled
1030,150,150_21,predictable,A,4.337975e-08,mastoids_pooled


In [8]:
conditions       = {("100", "random", "A-1"): "random/1/standard",
                    ("100", "random", "A-2"): "random/2/standard",
                    ("100", "random", "A-3"): "random/3/standard",
                    ("100", "random", "A-4"): "random/4/standard",
                    ("100", "random", "A-5"): "random/5/standard",
                    ("100", "random", "B-5"): "random/5/deviant",
                    ("150", "random", "A-1"): "random/1/standard",
                    ("150", "random", "A-2"): "random/2/standard",
                    ("150", "random", "A-3"): "random/3/standard",
                    ("150", "random", "A-4"): "random/4/standard",
                    ("150", "random", "A-5"): "random/5/standard",
                    ("150", "random", "B-5"): "random/5/deviant",

                    ("100", "predictable", "A-1"): "predictable/1/standard",
                    ("100", "predictable", "A-2"): "predictable/2/standard",
                    ("100", "predictable", "A-3"): "predictable/3/standard",
                    ("100", "predictable", "A-4"): "predictable/4/standard",
                    ("100", "predictable", "A-5"): "predictable/5/standard",
                    ("100", "predictable", "B-5"): "predictable/5/deviant",
                    
                    ("150", "predictable", "A-1"): "predictable/1/standard",
                    ("150", "predictable", "A-2"): "predictable/2/standard",
                    ("150", "predictable", "A-3"): "predictable/3/standard",
                    ("150", "predictable", "A-4"): "predictable/4/standard",
                    ("150", "predictable", "A-5"): "predictable/5/standard",
                    ("150", "predictable", "B-5"): "predictable/5/deviant"}

electrodes = {"FZ":"FZ", "CZ":"CZ", "M1": "M1", "M2": "M2", "fronto_pooled": ["FZ", "F3", "F4", "FC1", "FC2"], "mastoids_pooled": ["M1", "M2"]}

n1_window = (.140, .160)

amplitudes = [{ "SOA":key[0], "Participant":key[0] + "_" + str(i), 
                "Condition":key[1], "StimulusType":key[2], 
                "MeanAmplitude": amplitude,
                "Electrode": electrode} 
                    for key,value in conditions.items() 
                    for electrode, pick in electrodes.items()
                    for i, amplitude in enumerate(get_mean_amplitudes(evokeds_list_as_dict[key[0]][value], n1_window, picks=pick)) ]
amplitudes_df = pd.DataFrame(amplitudes)

amplitudes_df.to_csv("../data/n1_mean_amplitudes.csv", index=False)

amplitudes_df


,SOA,Participant,Condition,StimulusType,MeanAmplitude,Electrode
0,100,100_0,random,A-1,-1.530263e-07,FZ
1,100,100_1,random,A-1,-4.071953e-07,FZ
2,100,100_2,random,A-1,-1.431155e-06,FZ
3,100,100_3,random,A-1,1.515026e-07,FZ
4,100,100_4,random,A-1,-6.722830e-07,FZ
...,...,...,...,...,...,...
3091,150,150_18,predictable,B-5,1.457454e-06,mastoids_pooled
3092,150,150_19,predictable,B-5,1.019455e-07,mastoids_pooled
3093,150,150_20,predictable,B-5,-2.770085e-07,mastoids_pooled
3094,150,150_21,predictable,B-5,6.752592e-08,mastoids_pooled
